In [0]:
bronze_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/resource/origem/'
resource_path_volume = '/Volumes/infinitybigdata/azure/bikestore_resource/origem/'

In [0]:
%sql
--brands tabela física
CREATE TABLE if not EXISTS bikestore.logistics.bronze_brands
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/brands/'


In [0]:
'''
ETL com tabela temporária, geralmente nao se cria tabelas fisicas para a camada bronze, 
ela é ultilizada para o refinamento, o correto é criar tabelas fisicas na camada silver 
'''

In [0]:
# criando tabela temporária - para usar na proxima camada
(spark.read.format('delta')
  .load(f'{bronze_path}/customers/')
  .createOrReplaceTempView('tmp_custumers')
) 

In [0]:
# criar varias tabelas temporárias de forma prática
bronze_map = {
    "tmp_bronze_brands":      f"{bronze_path}/brands/",
    "tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_items/",
    "tmp_bronze_orders":      f"{bronze_path}/orders/",
    "tmp_bronze_products":    f"{bronze_path}/products/",
    "tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    "tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    "tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 

In [0]:
%sql
show tables 

Exemplo de criar tabels fisicas

In [0]:
%sql
-- brands
CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_brands
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/brands/';

-- categories
CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_categories
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/categories/';

-- customers
CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_customers
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/customers/';

-- order_items
CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_order_items
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/order_items/';

-- orders
CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_orders
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/orders/';

-- products
CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_products
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/products/';

-- staffs
CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_staffs
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/staffs/';

-- stocks
CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_stocks
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/stocks/';

-- stores
CREATE TABLE IF NOT EXISTS bikestore.logistics.bronze_stores
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/bikestore/bronze/stores/';


Criando uma camada Silver de teste


In [0]:
%sql
select * from tmp_bronze_products limit 10

In [0]:
%sql
select * from tmp_bronze_categories

In [0]:
%sql
describe tmp_bronze_categories

In [0]:
%sql
select
   p.product_id
  ,p.product_name
  ,p.brand_id
  --,p.category_id
  --,c.category_id as category_id_categoriat
  ,c.category_name
  ,p.model_year
  ,p.list_price
from tmp_bronze_products as p
left join tmp_bronze_categories as c on p.category_id = c.category_id

In [0]:
df_product_silver=spark.sql("""
select
   p.product_id
  ,p.product_name
  ,p.brand_id
  --,p.category_id
  --,c.category_id as category_id_categoriat
  ,c.category_name
  ,p.model_year
  ,p.list_price
from        tmp_bronze_products as p
left join tmp_bronze_categories as c on p.category_id = c.category_id

                            """) 

In [0]:
display (df_product_silver)

In [0]:
#salvar em parquet como delta  tabela de teste SILVER
df_product_silver.write\
    .mode('overwrite')\
    .format('delta')\
    .option('mergeSchema','true')\
    .save(f'{silver_path}/product_teste')

In [0]:
%sql
/*Criar tabela fisica de teste este comando prescisa rodar só 1x pois quando arquivos forem atualizados a tabela irá refletir o resultado  (irá pegar o log mais recente e trazer dados do arquivo(os) atualizados)*/
CREATE TABLE IF NOT EXISTS bikestore.logistics.products_teste
LOCATION 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/products_teste';

In [0]:
%sql
select * from bikestore.logistics.products_teste